# Interactive example of non numpy-able code optimization

In the debugging lecture last year we showed an example of a [not working] code to do the following:

* Consider a sequence of numbers a0, a1, ..., an, in which an element is equal to the sum of squared digits of the previous element. The sequence ends once an element that has already been in the sequence appears again. So that an = a0. Given the first element a0, find the length of the sequence.


So for example if a0 = 16. `1**2 + 6**2 = 37` -> `3**2 + 7**2 = 58` -> `5**2 + 8**2 = 89` -> `8**2 + 9**2 = 145` -> `1**2 + 4**2 + 5**2 = 42` -> `4**2 + 2**2 = 20` -> `2**2 + 0**2 = 4` -> `4**2 = 16` We've already seen 16 so we stop here. The sequence is `[16,37,58,89,145,42,20,4,16]` which has a length of 9, return 9.

The code that we used had some optimisations, but was not particularly clear. To try to understand it better let's write this out in a code that approaches the problem in a different way, with comments etc. to try to make it clearer what is going on

In [ ]:
def square_digits(input_number):

    # Initialize by setting the list of outputs equal to the input
    output_list = [input_number]
    # And setting the last_number variable to the input
    last_number = input_number

    # This is basically a for-loop that will only exit when we explicitly say "exit"
    while 1:
        # Step one: We must identify the digits of last_number
        # Cast to a string
        last_number = str(last_number)
        # And then convert to a list of integers. We do this using a list comprehension, which is powerful, but not fast
        digits = [int(digit) for digit in last_number]
        # So if last_number is 49120 then digits = [4,9,1,2,0]
        
        # We can then sum the digits squared using another list comprehension
        # digits = [4,9,1,2,0] -> 16 + 81 + 1 + 4 = 102
        digit_squared_sum = sum([digit*digit for digit in digits])
        
        # Is this value already in the list?
        if digit_squared_sum in output_list:
            # Add this value and then exist
            output_list.append(digit_squared_sum)
            break
        else:
            # Else add the value and then continue
            output_list.append(digit_squared_sum)
            last_number = digit_squared_sum

    return len(output_list)

print("square_digits(103) gives:",square_digits(103), "\n Should be 4")
print()
print("square_digits(612) gives:",square_digits(612), "\n Should be 16")
print()


In [ ]:
# Let's try it with the following
very_long_integer = 2**100
square_digits(very_long_integer)

Using the profiling tools demonstrated above:

* Use timeit to calculate how long the function takes to run with the `very_long_integer`
* Use lprun to determine how long the code spends at each line

Here is the optimized code that we used last year (with the bug fixed). As before run timeit and lprof on this code:

In [ ]:
def square_digits_withoutstr(input_number):

    cur = input_number
    was = set()

    while not (cur in was):
        was.add(cur)
        nxt = 0
        while cur > 0:
            nxt += (cur % 10) * (cur % 10)
            cur //= 10
        cur = nxt

    return len(was) + 1


In [ ]:
# Run timeit and lprun here


Now repeat the process using:
```very_long_integer=2**100000```

In [ ]:
# Run timeit and lprun here

very_long_integer = 2**100000


##  Summary

Think about what these results are telling you. Some things to highlight:

* You are learning how to identify which parts of a function you need to think about when optimising. Never bother optimising any part of your code that is not taking a significant fraction of the total time.
* The optimal solution to a problem can depend on the input. For shorter inputs, converting an integer to a string and then back to a list of integers adds an overhead that is the dominant computational cost. However, as the input integers become very large, operations like dividing by 10 (which for a computer is not as trivial as it seems, because computers think in binary numbers, not base-10 numbers) become expensive (look up how python handles big integers if you want to understand this better). In this case the overhead of converting to a string is faster.
* Although the second case was faster for normal-sized integers, it doesn't seem that using it would really be a good decision if the code is more difficult to parse.